In [67]:
from urllib.parse import urlencode
import os
import requests as re

import time

In [14]:
category = '/category?'
endpointpara = ['file_type','category_id']

In [21]:
def _build_endurl(parameter,endpointpara):
    para_all = {}
    for name,value in parameter['kwarg'].items():
        if name in endpointpara:
            para_all[name] = value
    
    para_all['api_key'] = 'b190fd48c808c7d3be5c40454f3e57f3'
    para_all['file_type'] = 'json'
    endurl = urlencode(para_all)

    return endurl

In [45]:
def _build_url(endpoint=None,endpointpara=None,map=None,**kwarg):
    if map:
        main_url = 'https://api.stlouisfed.org/geofred'
    else:
        main_url = 'https://api.stlouisfed.org/fred'
        
    end_url = _build_endurl(parameter=kwarg,endpointpara=endpointpara)

    return main_url + endpoint + end_url

In [56]:
def category_id(**kwarg):
    """ 
    category_id int : The id for a category.
    """
    # Define API Parameter
    endpoint = '/category?'
    endpointpara = ['file_type','category_id']
    # Contruct the url request
    print(kwarg)
    url_requst = _build_url(endpoint=endpoint,endpointpara=endpointpara,kwarg=kwarg)
    print(url_requst)
    try:
        result = re.get(url_requst)
        return result
    except Exception as error:
        print(f"An error occured '{error}'. Please check the parameter pass.")
        return None

In [63]:
def series_category(**kwarg):
    """
    category_id         int  : The id for a category.
    realtime_start      str  : The start of the real-time period. YYYY-MM-DD formatted string'
    realtime_end        str  : The end of the real-time period. YYYY-MM-DD formatted string'
    limit               int  : The maximum number of results to return
    offset              int  : offset postion of result
    order_by            str  : Order results by values of the specified attribute.
                                'series_id', 'title', 'units', 'frequency', 'seasonal_adjustment', 
                                'realtime_start', 'realtime_end', 'last_updated', 'observation_start', 
                                'observation_end', 'popularity', 'group_popularity'.
    sort_order          str  : Sort results is ascending or descending One of the following strings: 'asc', 'desc'.
    filter_variable     str  : The attribute to filter results by.'frequency', 'units', 'seasonal_adjustment'.
    filter_value        str  : The value of the filter_variable attribute to filter results by.
    tag_names           str  : A semicolon delimited list of tag names that series match all of. 
                                Example value: 'income;bea'. Filter results to series having both tags 'income' and 'bea'.
    exclude_tag_names   str  : A semicolon delimited list of tag names that series match none of.
                                Example value: 'discontinued;annual'. Filter results to series having neither tag 'discontinued' nor tag 'annual'.
    """
    # Define API Parameter
    endpoint = '/category/series?'
    endpointpara = ['file_type','category_id','realtime_start','realtime_end','limit','offset','order_by','sort_order','filter_variable','filter_value','tag_names','exclude_tag_names']
    # Contruct the url request    
    url_requst = _build_url(endpoint=endpoint,endpointpara=endpointpara,kwarg=kwarg)
    try:
        result = re.get(url_requst)
        return result
    except Exception as error:
        print(f"An error occured '{error}'. Please check the parameter pass.")
        return None

In [60]:
result = category_id(map=True, category_id=125,limit=1000,offset=10)
json = result.json()

{'map': True, 'category_id': 125, 'limit': 1000, 'offset': 10}
https://api.stlouisfed.org/fred/category?category_id=125&api_key=b190fd48c808c7d3be5c40454f3e57f3&file_type=json


In [59]:
json

{'categories': [{'id': 125, 'name': 'Trade Balance', 'parent_id': 13}]}

In [65]:
result = series_category(map=None, category_id=125,limit=1000,offset=10)
json = result.json()
json

{'realtime_start': '2024-05-10',
 'realtime_end': '2024-05-10',
 'order_by': 'series_id',
 'sort_order': 'asc',
 'count': 47,
 'offset': 10,
 'limit': 1000,
 'seriess': [{'id': 'BOPBIIN',
   'realtime_start': '2024-05-10',
   'realtime_end': '2024-05-10',
   'title': 'Balance on Investment Income (DISCONTINUED)',
   'observation_start': '1960-01-01',
   'observation_end': '2014-01-01',
   'frequency': 'Quarterly',
   'frequency_short': 'Q',
   'units': 'Billions of Dollars',
   'units_short': 'Bil. of $',
   'seasonal_adjustment': 'Not Seasonally Adjusted',
   'seasonal_adjustment_short': 'NSA',
   'last_updated': '2014-06-18 08:41:27-05',
   'popularity': 0,
   'group_popularity': 1,
   'notes': 'This series has been discontinued as a result of the comprehensive restructuring of the international economic accounts (https://apps.bea.gov/scb/pdf/2014/07%20July/0714_annual_international_transactions_accounts.pdf). For a crosswalk of the old and new series in FRED see: http://research.stl

In [68]:
def getseriesid(id):
    offset = 0
    item = 1
    all_list = []
    while  offset<item:
        result = series_category(category_id=id,limit=1000,offset=offset)

        if result is None:
            print('Result return with Noting skipping current call.')
            offset+= 1000

        elif result.ok:
            json = result.json()
            item =json['count']
            offset+= 1000
            all_list.extend(json['seriess'])
            
        else:
            print(f"Issue while calling API {result.status_code}")
            print(result.content)
            time.sleep(5)
    count_data = len(all_list)
    print(f'Done for ID {id} get total of {count_data}')
    time.sleep(1)
    return all_list,count_data

In [71]:
result_id,count_data = getseriesid(125)

Done for ID 125 get total of 47


In [74]:
all_list = []
id_data = {}

In [76]:
all_list.extend(result_id)
id_data[125] = count_data

In [78]:
import pandas as pd
df = pd.json_normalize(all_list)

In [79]:
df.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
0,AITGCBN,2024-05-10,2024-05-10,Advance U.S. International Trade in Goods: Bal...,2024-03-01,2024-03-01,Monthly,M,Millions of Dollars,Mil. of $,Not Seasonally Adjusted,NSA,2024-04-25 07:31:06-05,1,24,This advance estimate represents the current m...
1,AITGCBS,2024-05-10,2024-05-10,Advance U.S. International Trade in Goods: Bal...,2024-03-01,2024-03-01,Monthly,M,Millions of Dollars,Mil. of $,Seasonally Adjusted,SA,2024-04-25 07:31:02-05,24,24,This advance estimate represents the current m...
2,BOPBCA,2024-05-10,2024-05-10,Balance on Current Account (DISCONTINUED),1960-01-01,2014-01-01,Quarterly,Q,Billions of Dollars,Bil. of $,Seasonally Adjusted,SA,2014-06-18 08:41:28-05,15,16,This series has been discontinued as a result ...
3,BOPBCAA,2024-05-10,2024-05-10,Balance on Current Account (DISCONTINUED),1960-01-01,2013-01-01,Annual,A,Billions of Dollars,Bil. of $,Not Seasonally Adjusted,NSA,2014-06-18 08:41:28-05,2,16,This series has been discontinued as a result ...
4,BOPBCAN,2024-05-10,2024-05-10,Balance on Current Account (DISCONTINUED),1960-01-01,2014-01-01,Quarterly,Q,Billions of Dollars,Bil. of $,Not Seasonally Adjusted,NSA,2014-06-18 08:41:28-05,1,16,This series has been discontinued as a result ...
